In [1]:
import os
import shutil
import sys

city = "kl"
extract = "https://s3.amazonaws.com/metro-extracts.mapzen.com/kuala-lumpur_malaysia.osm.pbf"
output_data_dir = "/data/tiles/"


# need to use local file system to build tiles -- docker mounted volumes are too slow
tmp_build_dir = "/tmp/tiles/"

city_tmp_dir = tmp_build_dir + city + "/"

if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)

city_output_dir = output_data_dir + city + "/"
    
if os.path.exists(city_output_dir):
    print "Output file " + city_output_dir + " already exists."
    sys.exit()

os.makedirs(city_output_dir)

if os.path.exists(city_tmp_dir):
    shutil.rmtree(city_tmp_dir)
    
os.makedirs(city_tmp_dir)
os.chdir(city_tmp_dir)

# download some data and make tiles out of it
print "Downloading: " + extract
os.system("wget " + extract + " -O extract.pbf")

#get the config and setup for it
print "Building tile config..."
os.system("valhalla_build_config --mjolnir-tile-dir " + city_tmp_dir + "valhalla_tiles --mjolnir-tile-extract " + city_tmp_dir + "valhalla_tiles.tar --mjolnir-timezone " + city_tmp_dir + "valhalla_tiles/timezones.sqlite --mjolnir-admin " + city_tmp_dir + "valhalla_tiles/admins.sqlite > " + city_tmp_dir + "valhalla.json")

#build routing tiles
print "Buiding base tiles..."
os.system("valhalla_build_tiles -c valhalla.json extract.pbf")

#tar it up for running the server
print "Buiding tar of tiles..."
os.system("find valhalla_tiles | sort -n | tar cf valhalla_tiles.tar --no-recursion -T -")

#make some osmlr segments
print "Buiding OSMLR tiles..."
os.system("LD_LIBRARY_PATH=/usr/lib:/usr/local/lib osmlr -m 1 -T " + city_tmp_dir +"osmlr_tiles valhalla.json")

# -j 2 uses two threads for association process (use more or fewer as available cores permit
print "Associate OSMLR tiles..."
os.system("valhalla_associate_segments -t " + city_tmp_dir + "osmlr_tiles -j 2 --config valhalla.json")

# rebuild tar with traffic segement associated tiles
print "Buiding osmlr tile tar..."
os.system("find valhalla_tiles | sort -n | tar rf associated_tiles.tar --no-recursion -T -")

#make some osmlr segments -- this time in geojson format...
print "Buiding OSMLR tiles..."
os.system("LD_LIBRARY_PATH=/usr/lib:/usr/local/lib osmlr  -J " + city_tmp_dir +"osmlr_geotiles valhalla.json")

print "Moving built tar tile to " + city_output_dir
shutil.move(city_tmp_dir + "associated_tiles.tar", city_output_dir + "tiles.tar")
shutil.move(city_tmp_dir + "osmlr_geotiles", city_output_dir + "osmlr_geotiles")


Downloading: https://s3.amazonaws.com/metro-extracts.mapzen.com/kuala-lumpur_malaysia.osm.pbf
Building tile config...
Buiding base tiles...
Buiding tar of tiles...
Buiding OSMLR tiles...
Associate OSMLR tiles...
Buiding osmlr tile tar...
Buiding OSMLR tiles...
Moving built tar tile to /data/tiles/kl/
